In [1]:
import os, sys
from dotenv import find_dotenv
from datetime import timedelta

path_to_utils_module = os.path.join(
    os.path.dirname(find_dotenv()), "..", "integration_tests", "tests"
)
sys.path.insert(0, path_to_utils_module)
from utils import make_token
# Create an API access token

claims = {
    "daily_location": {
        "permissions": {"run": True, "poll": True, "get_result": True},
        "spatial_aggregation": ["admin3", "admin2"],
    },
    "modal_location": {
        "permissions": {"run": True, "poll": True, "get_result": True},
        "spatial_aggregation": ["admin3", "admin2"],
    },
    "flows": {
        "permissions": {"run": True, "poll": True, "get_result": True},
        "spatial_aggregation": ["admin3", "admin2", "admin1"],
    },
    "location_event_counts": {
        "permissions": {"run": True, "poll": True, "get_result": True},
        "spatial_aggregation": ["admin3", "admin2", "admin1"],
    },
    "meaningful_locations_aggregate": {
        "permissions": {"run": True, "poll": True, "get_result": True},
        "spatial_aggregation": ["admin3", "admin2", "admin1"],
    },
    "meaningful_locations_od_matrix": {
        "permissions": {"run": True, "poll": True, "get_result": True},
        "spatial_aggregation": ["admin3", "admin2", "admin1"],
    },
    "geography": {
        "permissions": {"run": True, "poll": True, "get_result": True},
        "spatial_aggregation": ["admin3", "admin2", "admin1"],
    }
}

TOKEN = make_token(
    username="testuser", secret_key="secret", lifetime=timedelta(days=1), claims=claims
)

In [2]:
import flowclient, folium

In [21]:
conn = flowclient.connect("http://localhost:5000", TOKEN)

/Users/jono/code/FlowKit/flowclient/flowclient/client.py:200: UserWarning: Communications with this server are NOT SECURE.
  return Connection(url, token, api_version, ssl_certificate)


In [4]:
regions = flowclient.get_geography(conn, "admin3")

In [27]:
meaningful_locations_spec = flowclient.meaningful_locations_aggregate("2016-01-01", "2016-01-07", "evening", labels={
                    "evening": {
                        "type": "Polygon",
                        "coordinates": [
                            [[1, 1], [1, -1], [1e-06, -1], [1e-06, 1]]
                        ],
                    },
                    "day": {
                        "type": "Polygon",
                        "coordinates": [
                            [[0, 1], [0, -1], [-1, -1], [-1, 1]]
                        ],
                    },
                }, tower_day_of_week_scores={
                    "monday": 1,
                    "tuesday": 1,
                    "wednesday": 1,
                    "thursday": 0,
                    "friday": -1,
                    "saturday": -1,
                    "sunday": -1,
                }, tower_hour_of_day_scores=[
                    -1,
                    -1,
                    -1,
                    -1,
                    -1,
                    -1,
                    -1,
                    0,
                    0,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    0,
                    0,
                    0,
                    0,
                    -1,
                    -1,
                    -1,
                ], aggregation_unit="admin3")

In [ ]:
flowclient.get_result(conn, meaningful_locations_spec)

In [18]:
flowclient.get_result(conn, meaningful_locations_spec)

,label,pcod,total
0,day,524 1 02 09,16.0
1,day,524 4 12 62,20.0
2,day,524 4 12 66,16.0


In [24]:
meaningful_locations_spec = flowclient.meaningful_locations_between_label_od_matrix("2016-01-01", "2016-01-07", "evening", "day", labels={
                    "evening": {
                        "type": "Polygon",
                        "coordinates": [
                            [[1, 1], [1, -1], [1e-06, -1], [1e-06, 1]]
                        ],
                    },
                    "day": {
                        "type": "Polygon",
                        "coordinates": [
                            [[0, 1], [0, -1], [-1, -1], [-1, 1]]
                        ],
                    },
                }, tower_day_of_week_scores={
                    "monday": 1,
                    "tuesday": 1,
                    "wednesday": 1,
                    "thursday": 0,
                    "friday": -1,
                    "saturday": -1,
                    "sunday": -1,
                }, tower_hour_of_day_scores=[
                    -1,
                    -1,
                    -1,
                    -1,
                    -1,
                    -1,
                    -1,
                    0,
                    0,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    0,
                    0,
                    0,
                    0,
                    -1,
                    -1,
                    -1,
                ], aggregation_unit="admin3")

In [26]:
flowclient.get_result(conn, meaningful_locations_spec)

""
